In [1]:
from __future__ import print_function

import os
import mat73
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as scio
import torch
import torch.optim
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim

from architecture import MST
torch.cuda.manual_seed(seed=666)
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor
device = torch.device('cuda')
torch.cuda.manual_seed(seed=666)


########################################### 辅助函数 ###########################################
def thres_21(L, tau, M, N, B):
    S = torch.sqrt(torch.sum(torch.mul(L, L), 2))
    S[S == 0] = 1
    T = 1 - tau / S
    T[T < 0] = 0
    R = T.reshape(M, N, 1).repeat((1, 1, B))
    res = torch.mul(R, L)
    return res


################################ 读取数据 ################################
YY= mat73.loadmat("./Data/T31UDQ/T31UDQ_20m_raw1.mat")['T31UDQ_20m_raw1'] 
YY = np.double(YY)
YY = YY/YY.max()

Mask2_500 = mat73.loadmat("./Data/Mask2_500.mat")['Mask2_500']
Mask_Out_500 = mat73.loadmat("./Data/Mask_Out_500.mat")["Mask_Out_500"]
Mask_Shd = mat73.loadmat("./Data/Mask_Shd.mat")["Mask_Shd"]
Mask_shad = mat73.loadmat("./Data/Mask_shad.mat")["Mask_shad"]
Mask = np.copy(Mask2_500)
Mask[ Mask != 0 ] = 1
# Test1_Clean是干净图像，取的是4，2，0三个时相
Test1_Clean = np.copy(YY[:, :, :, [4, 2, 0]])

# 生成云图YY，含有6个时相
Clean = YY[:, :, :, [4, 2, 0, 5, 3, 1]]
YY = np.multiply((1 - Mask), (YY[:, :, :, [4, 2, 0, 5, 3, 1]])) + Mask
YY = np.multiply((1 - Mask_Out_500),  YY) + Mask_Out_500
YY = np.multiply(YY, (1 - Mask_Shd) ) - Mask_Shd
YY = np.multiply(YY, (1 - Mask_shad) ) - Mask_shad
Noisy = np.copy(YY)
Cloud = np.copy(Mask)
Mask = 1 - Cloud
###########################################################################

M, N, B, T = Noisy.shape
Clean = torch.from_numpy(Clean.reshape(M, N, B, T)).to(device)
Mask = torch.from_numpy(Mask.reshape(M, N, B, T)).to(device)  # 云=0
Cloud = torch.from_numpy(Cloud.reshape(M, N, B, T)).to(device)  # 云=1
Noisy = torch.from_numpy(Noisy.reshape(M, N, B, T)).to(device)


################################ 全局设置 ################################
def grid_combine(args_list):
    for rate in args_list[0]:
        for lambda2 in args_list[1]:
            for rho in args_list[2]:
                for alpha in args_list[3]:
                    if rate == 125 and rho ==0.08 and alpha==1.03 :
                        continue
                    yield rate, lambda2, rho, alpha
args_list = [
    [125, 100],  # rate
    [0.1],  # lambda2
    [0.08, 0.06, 0.04],  # rho
    [1.03, 1.025]  # alpha
]
condidates = [
    # lambda1, lambda2, rho
    # [6.0, 0.1, 0.12, 1.03]     # condidate2
    # [12.5, 0.1, 0.1], 
    # [10.0, 0.1, 0.1], 
    # [7.50, 0.1, 0.1],  
    # [10.0, 0.2, 0.1], 
    # [10.0, 0.4, 0.1], 
]
condidates = [
    # rate, lambda2, rho, alpha
    # [90, 0.06, 0.06, 1.03],
    [90, 0.08, 0.08, 1.03],
    [88, 0.06, 0.08, 1.03],
    [90, 0.08, 0.08, 1.03],
]
iter_num, epoch_num = 200, 300
order = 15
for rate, lambda2, rho, alpha in condidates:
# for rate, lambda2, rho, alpha in grid_combine(args_list):
    # rate = lambda1 / rho
    lambda1 = rate * rho
    iters = 0
    order += 1
    log_dir = "./log-candidates-picardie/order[%03d]-iter[%3d]-epoch[%3d]-rate[%.3f]-lambda2[%.3f]-rho[%.3f,%.3f]" \
              % (order, iter_num, epoch_num, rate, lambda2, rho, alpha)
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    else:
        continue
    # rate, lambda2, rho, alpha = 125, 0.1, 0.08, 1.04
    ################################ 初始化 ################################
    Y = Noisy.clone()
    X = Noisy.clone()
    W = Noisy.clone()
    C = torch.zeros(Y.shape).type(dtype).to(device)
    M_updata = Mask.clone()
    model = MST(dim=B*T, stage=2, num_blocks=[2, 2, 2]).cuda()
    psnr = 0
    s0_min = 10000
    ################################ 子问题更新 ################################
    while iters < iter_num:
        # if iters >= 100 and psnr < 32:
        #     break
        iters += 1
        ################################ 更新子问题X ################################
        # if decrease and (iters % 4 == 0) and LR > 0.00001:
        #     LR *= 0.9
        LR = 0.0002
        optimizer = torch.optim.Adam(model.parameters(), lr=LR, betas=(0.9, 0.999))
        # scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [250], gamma=0.7, last_epoch=-1)
        loss_history, loss1_history, loss2_history = [], [], []
        epoch = 0
        net_input = torch.permute(X, [2, 3, 0, 1]).reshape(1, B * T, M, N).type(dtype).to(device)
        while epoch < epoch_num:
            epoch += 1
            optimizer.zero_grad()
            # X: M, N, B, T
            Mask_temp = torch.permute(M_updata, [2, 3, 1, 0]).reshape(1, B*T, M, N).type(dtype).to(device)
            out = model(net_input, Mask_temp)
            X = torch.permute(out.reshape(B, T, M, N), [2, 3, 0, 1])
            loss1 = 1 / 2 * torch.norm(Y - torch.mul(X, M_updata) - C, 'fro') ** 2
            loss2 = rho / 2 * torch.norm(X - W, 'fro') ** 2
            loss = loss1 + loss2
            loss.backward()
            optimizer.step()
            # scheduler.step()
            loss_history.append(loss.item())
            loss1_history.append(loss1.item())
            loss2_history.append(loss2.item())
            if epoch % 40 == 0:
                print("\tThe %03dth iters, the %03dth epoch, loss: %.5f" % (iters, epoch, loss.item()))
            Y, C, W, X = Y.detach(), C.detach(), W.detach(), X.detach()

        Y, C, W, X = Y.detach(), C.detach(), W.detach(), X.detach()

        ################################ 更新子问题W ################################
        U, s, VH = torch.linalg.svd(X.reshape(M * N, B * T), full_matrices=False)  # B*T, M*N
        s0 = s[0]
        rho *= alpha
        # if s0<s0_min: # s0处于不断下降的第一阶段
        #     s0_min = s0
        #     rho *= alpha
        # else:
        #     rho *= 1.02
        rate = lambda1 / rho
        print("s: ", s)
        s = s - lambda1 / rho
        s[s < 0] = 0
        S = torch.diag(s)
        W = torch.mm(torch.mm(U, S), VH).reshape(M, N, B, T)
        ################################ 更新子问题C ################################
        L = Y - torch.mul(X, M_updata)
        for t in range(T):
            C[:, :, :, t] = thres_21(L[:, :, :, t], lambda2, M, N, B)

        ################################ 更新掩码M ################################
        B_mean = torch.mean(Y-X, dim=2).reshape(M, N, 1, T).repeat((1, 1, B, 1))
        M_updata[:, :, :, :3][torch.abs(B_mean[:, :, :, :3]) > 0.01] = 0   # M_update=1表示无云，M_update=0表示云
        if iters > 5:
            M_updata[:, :, :, :3][torch.abs(B_mean[:, :, :, :3]) < 0.01] = 1

        ################################ 进行指标评估 ################################
        # mse = torch.sum((X - Clean) ** 2)
        psnr = []
        for t in range(3):
            psnr_t = 0
            for b in range(B):
                psnr_t += compare_psnr(X[:, :, b, t].detach().cpu().numpy(), Clean[:, :, b, t].detach().cpu().numpy(), data_range=1)
            psnr.append(psnr_t / B)
        ssim = []
        for t in range(3):
            ssim_b = 0
            for b in range(B):
                ssim_b += compare_ssim(X[:, :, b, t].detach().cpu().numpy(), Clean[:, :, b, t].detach().cpu().numpy())
            ssim.append(ssim_b / B)
        # print("The %03dth iters, the %03dth epoch, loss: %.5f, mse: %.5f, psnr: %.5f, ssim: %.5f" % (
        #     iters, epoch, loss.item(), mse, psnr, ssim))

        image_Clean = Clean.cpu().numpy()
        image_Y = Y.cpu().numpy()
        image_X = X.cpu().detach().numpy()
        image_C = C.cpu().numpy()
        image_M = M_updata.cpu().numpy()
        plt.figure(figsize=(20, 20))
        for i in range(6):
            plt.subplot(6, 6, 1 + i)
            plt.title("Clean")
            plt.imshow(image_Clean[:, :, [4, 2, 0], i]*5)
            plt.axis('off')

            plt.subplot(6, 6, 7 + i)
            plt.title("X")
            plt.imshow(image_X[:, :, [4, 2, 0], i]*5)
            plt.axis('off')

            plt.subplot(6, 6, 13 + i)
            plt.title("Y")
            plt.imshow(image_Y[:, :, [4, 2, 0], i]*5)
            plt.axis('off')

            plt.subplot(6, 6, 19 + i)
            plt.title("Y-X")
            plt.imshow(image_Y[:, :, [4, 2, 0], i]*5 - image_X[:, :, [4, 2, 0], i]*5)
            plt.axis('off')

            plt.subplot(6, 6, 25 + i)
            plt.title("C")
            plt.imshow(image_C[:, :, 0, i]*5, cmap='gray')
            plt.axis('off')

            plt.subplot(6, 6, 31 + i)
            plt.title("M_update")
            plt.imshow(image_M[:, :, 0, i], cmap='gray')
            plt.axis('off')
        psnr1, psnr2, psnr3 = psnr
        ssim1, ssim2, ssim3 = ssim
        result_path = "%s/iter[%03d, %03d]-pnsr[%.3f, %.3f, %.3f]-ssim[%.3f, %.3f, %.3f].png" % (
            log_dir, iters, epoch, psnr1, psnr2, psnr3, ssim1, ssim2, ssim3)
        plt.savefig(result_path)
        plt.clf()
        plt.figure(figsize=(30, 60))
        plt.subplot(6, 1, 1)
        plt.title("total loss")
        plt.plot([i for i in range(len(loss_history[50:]))], loss_history[50:])

        plt.subplot(6, 1, 2)
        plt.title("loss1")
        plt.plot([i for i in range(len(loss1_history[50:]))], loss1_history[50:])

        plt.subplot(6, 1, 3)
        plt.title("loss2")
        plt.plot([i for i in range(len(loss2_history[50:]))], loss2_history[50:])

        plt.subplot(6, 1, 4)
        plt.title("total loss")
        plt.plot([i for i in range(len(loss_history))], loss_history)

        plt.subplot(6, 1, 5)
        plt.title("loss1")
        plt.plot([i for i in range(len(loss1_history))], loss1_history)

        plt.subplot(6, 1, 6)
        plt.title("loss2")
        plt.plot([i for i in range(len(loss2_history))], loss2_history)

        loss_path = "%s/iter[%03d, %03d]-LR[%f]-rate[%.3f]-s0[%.3f]-rho[%.3f]-loss[%.3f].png" % (
            log_dir, iters, epoch, LR, rate, s0, rho, loss.item())
        plt.savefig(loss_path)
        plt.clf()

        plt.close()
    # decrease = True
    # up = False


	The 001th iters, the 040th epoch, loss: 579114.25622
	The 001th iters, the 080th epoch, loss: 578907.67101
	The 001th iters, the 120th epoch, loss: 578862.84496
	The 001th iters, the 160th epoch, loss: 578838.11577
	The 001th iters, the 200th epoch, loss: 578821.92171
	The 001th iters, the 240th epoch, loss: 578810.23776
	The 001th iters, the 280th epoch, loss: 578801.38513
s:  tensor([941.3847, 630.8392, 472.9757, 111.1894,  30.0804,  12.1586,   8.0623,
          7.8956,   7.0650,   6.8323,   5.6556,   4.6011,   3.8349,   3.2650,
          3.1383,   2.8897,   2.6637,   2.4674,   2.3870,   2.3327,   2.2411,
          2.2126,   2.1094,   1.9968,   1.9501,   1.8703,   1.8102,   1.7287,
          1.7012,   1.6673,   1.6337,   1.5699,   1.5231,   1.4751,   1.3667,
          1.3186], device='cuda:0')


d:\development\anaconda\envs\deep-todo\lib\site-packages\skimage\_shared\utils.py:348: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
  return func(*args, **kwargs)
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the va

	The 002th iters, the 040th epoch, loss: 2136.45024
	The 002th iters, the 080th epoch, loss: 1968.03405
	The 002th iters, the 120th epoch, loss: 1877.59669
	The 002th iters, the 160th epoch, loss: 1820.71346
	The 002th iters, the 200th epoch, loss: 1783.65917
	The 002th iters, the 240th epoch, loss: 1759.36641
	The 002th iters, the 280th epoch, loss: 1728.70807
s:  tensor([866.7880, 569.7719, 437.6391, 103.9275,  29.7354,  12.1348,   8.0048,
          7.8371,   7.1036,   6.8867,   5.7584,   4.7052,   3.9263,   3.6277,
          3.4213,   3.3512,   3.1657,   2.9890,   2.9679,   2.8173,   2.6672,
          2.6172,   2.5124,   2.4218,   2.3682,   2.3436,   2.2161,   2.1602,
          2.1207,   2.0760,   2.0377,   1.9924,   1.9582,   1.7844,   1.7588,
          1.6934], device='cuda:0')


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

	The 003th iters, the 040th epoch, loss: 100341.27839
	The 003th iters, the 080th epoch, loss: 100281.51585
	The 003th iters, the 120th epoch, loss: 100253.43556
	The 003th iters, the 160th epoch, loss: 100237.79560
	The 003th iters, the 200th epoch, loss: 100219.74418
	The 003th iters, the 240th epoch, loss: 100210.90739
	The 003th iters, the 280th epoch, loss: 100199.67297
s:  tensor([790.0441, 498.6527, 381.7719, 102.1763,  29.1891,  12.0216,   7.8895,
          7.7269,   7.1501,   6.9471,   5.7977,   5.0378,   4.1388,   4.0242,
          3.9544,   3.6913,   3.6125,   3.5144,   3.3216,   3.1795,   3.1228,
          2.9395,   2.9312,   2.8175,   2.7647,   2.7276,   2.5677,   2.4437,
          2.4162,   2.3324,   2.2615,   2.2537,   2.2159,   2.0585,   2.0225,
          1.9802], device='cuda:0')


...

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

In [ ]:
import torch
a = torch.ones((10, 10, 10))
# a[:,:5,:][a[:, :5, :] == 10] = 1
a[0, 1, 0]
index = torch.where(a[:, :5, :] == 10, a[:, :5, :], a[:, :5, :]-1)
index[0, 0, 0]


In [ ]:
_, a, _ = torch.linalg.svd(X.reshape(M*N, B*T), full_matrices=False)  # B*T, M*N
a